In [1]:
import data_mining as dm
from lstm_autoencoder import RecurrentAutoencoder
import train
import numpy as np
import torch
import importlib
import pandas as pd
from torch import nn

from get_paquetes import capturar_paquetes
import clean_paquetes


In [2]:
device = torch.device("cpu")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

Get package

In [3]:
num_paquetes = 25
datos_capturados = capturar_paquetes(num_paquetes)

#convertir a df
#poner nombres a las columnas times, src_ips, dst_ips, protocols, lengths
df = pd.DataFrame(datos_capturados, columns = ['Time', 'Source', 'Destination', 'Protocol', 'Length'])
#add a column of info and all values are '1'
df_info = pd.DataFrame(np.ones((len(df),1)), columns = ['Info'])
df_no= pd.DataFrame(np.ones((len(df),1)), columns = ['No.'])
df = pd.concat([df, df_info], axis=1)
df = pd.concat([df_no, df], axis=1)

#print(df)

#limpiar los datos
train_dataset,seq_len,n_features = clean_paquetes.clean_and_get_Data(df,25)
#print(df)


True


c:\Users\flaco\OneDrive\Escritorio\hackaton_2024\Cybersecurity\clean_paquetes.py:55: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Source'] = df['Source'].str.replace('.', '')
c:\Users\flaco\OneDrive\Escritorio\hackaton_2024\Cybersecurity\clean_paquetes.py:56: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Destination'] = df['Destination'].str.replace('.', '')


(25, 3)
el shape del dataset torch.Size([14, 10, 3, 1])
14 10 3


In [4]:
def predict(model, dataset):
  predictions, losses = [], []
  criterion = nn.L1Loss(reduction='sum').to(device)
  with torch.no_grad():
    model = model.eval()
    for seq_true in dataset:
      seq_true=seq_true.reshape((25,1,3))
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      seq_pred=seq_pred.reshape((25,1,3))
      loss = criterion(seq_pred, seq_true)

      predictions.append(seq_pred.cpu().numpy().flatten())
      losses.append(loss.item())
  return predictions,losses

In [9]:
#cargamos los pesos del modelo lstm autoencoder
from torch import nn
model_path = '25model.pth'
#loaded_model = torch.load(model_path)
loaded_model = torch.load('25model.pth', map_location=torch.device('cpu'))

#convert to numpy
train_dataset = np.ndarray(train_dataset)
print(train_dataset.shape)
losses= predict(loaded_model, train_dataset)


TypeError: only integer tensors of a single element can be converted to an index